### Connect to DuckDB (Day-11)

In [1]:
from pathlib import Path
import duckdb
import pandas as pd

def find_repo_root(start: Path | None = None) -> Path:
    p = (start or Path.cwd()).resolve()
    while True:
        if (p / "Day-1").exists() and (p / "Day-11").exists():
            return p
        if p == p.parent:
            raise FileNotFoundError("Could not locate repo root. Expected Day-1 and Day-11.")
        p = p.parent

repo_root = find_repo_root()
db_path = repo_root / "Day-11" / "data" / "warehouse" / "day11_noshow.duckdb"

print("Repo root:", repo_root)
print("DB path  :", db_path)
print("Exists   :", db_path.exists())

con = duckdb.connect(str(db_path))
print("Tables:", [t[0] for t in con.execute("SHOW TABLES").fetchall()])


Repo root: C:\Users\sarfo\Dropbox\Courses\Data Science\30-days-of-data-science
DB path  : C:\Users\sarfo\Dropbox\Courses\Data Science\30-days-of-data-science\Day-11\data\warehouse\day11_noshow.duckdb
Exists   : True
Tables: ['bronze_appointments', 'gold_appointments_base', 'silver_appointments']


### Inspect available columns

In [2]:
cols = con.execute("PRAGMA table_info('gold_appointments_base')").df()
cols[["name","type"]]


,name,type
0,appointment_id,BIGINT
1,person_id,BIGINT
2,scheduled_ts,TIMESTAMP
3,appointment_ts,TIMESTAMP
4,scheduled_date,DATE
5,appointment_date,DATE
6,lead_time_days,INTEGER
7,date_inversion_flag,INTEGER
8,age,DOUBLE
9,gender,VARCHAR


### Build gold_appointments_features_v1 in DuckDB

In [3]:
con.execute("""
CREATE OR REPLACE TABLE gold_appointments_features_v1 AS
WITH base AS (
    SELECT
        appointment_id,
        person_id,
        label,
        sms_received,

        age,
        gender,
        neighbourhood,
        scholarship,
        hipertension,
        diabetes,
        alcoholism,
        handcap,

        lead_time_days,

        -- Use date columns if they exist; if timestamp exists, cast to DATE
        CASE
            WHEN try_cast(appointment_date AS DATE) IS NOT NULL THEN try_cast(appointment_date AS DATE)
            ELSE try_cast(appointment_ts AS DATE)
        END AS appt_date,

        CASE
            WHEN try_cast(scheduled_date AS DATE) IS NOT NULL THEN try_cast(scheduled_date AS DATE)
            ELSE try_cast(scheduled_ts AS DATE)
        END AS sched_date,

        -- If timestamp exists, extract hour; else NULL
        CASE
            WHEN try_cast(scheduled_ts AS TIMESTAMP) IS NOT NULL THEN extract('hour' FROM try_cast(scheduled_ts AS TIMESTAMP))
            ELSE NULL
        END AS sched_hour
    FROM gold_appointments_base
),
time_feats AS (
    SELECT
        *,
        extract('dow' FROM appt_date) AS appt_dow,   -- 0=Sunday in DuckDB
        extract('month' FROM appt_date) AS appt_month,
        extract('dow' FROM sched_date) AS sched_dow,
        extract('month' FROM sched_date) AS sched_month
    FROM base
),
lead_feats AS (
    SELECT
        *,
        -- Clip to avoid extreme values driving the model
        GREATEST(-1, LEAST(lead_time_days, 180)) AS lead_time_clipped,

        -- Log transform (shift so 0 stays 0)
        ln(1 + GREATEST(0, lead_time_days)) AS lead_time_log1p,

        CASE
            WHEN lead_time_days <= 0 THEN 'same_day'
            WHEN lead_time_days <= 2 THEN '1_2'
            WHEN lead_time_days <= 7 THEN '3_7'
            WHEN lead_time_days <= 14 THEN '8_14'
            WHEN lead_time_days <= 30 THEN '15_30'
            WHEN lead_time_days <= 60 THEN '31_60'
            ELSE '61_plus'
        END AS lead_time_bin
    FROM time_feats
),
nbhd_counts AS (
    SELECT neighbourhood, COUNT(*) AS nbhd_n
    FROM lead_feats
    GROUP BY 1
),
nbhd_feats AS (
    SELECT
        lf.*,
        nc.nbhd_n,
        CASE
            WHEN nc.nbhd_n < 200 THEN 'OTHER_SMALL'
            ELSE lf.neighbourhood
        END AS neighbourhood_grp
    FROM lead_feats lf
    LEFT JOIN nbhd_counts nc
    USING (neighbourhood)
),
-- Patient prior history using appointment date ordering (no look-ahead)
hist AS (
    SELECT
        *,
        COUNT(*) OVER (
            PARTITION BY person_id
            ORDER BY appt_date, appointment_id
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS prior_appt_count
    FROM nbhd_feats
)
SELECT
    appointment_id,
    person_id,
    label,
    sms_received,

    age,
    gender,
    neighbourhood_grp AS neighbourhood,
    scholarship, hipertension, diabetes, alcoholism, handcap,

    lead_time_days,
    lead_time_clipped,
    lead_time_log1p,
    lead_time_bin,

    appt_date,
    sched_date,
    appt_dow,
    appt_month,
    sched_dow,
    sched_month,
    sched_hour,

    nbhd_n,
    prior_appt_count
FROM hist
""")

print("Built: gold_appointments_features_v1")


Built: gold_appointments_features_v1


### Sanity checks on the new table

In [4]:
n = con.execute("SELECT COUNT(*) FROM gold_appointments_features_v1").fetchone()[0]
prev = con.execute("SELECT AVG(label) FROM gold_appointments_features_v1").fetchone()[0]
print("rows:", n)
print("no-show prevalence:", prev)

dup = con.execute("""
SELECT COUNT(*) - COUNT(DISTINCT appointment_id)
FROM gold_appointments_features_v1
""").fetchone()[0]
print("duplicate appointment_id:", dup)


rows: 110516
no-show prevalence: 0.20188027073003004
duplicate appointment_id: 0


### Check leakage risk for history feature

We only used “prior appointments” strictly before current row—good. Let’s confirm it’s non-negative and not crazy:

In [5]:
con.execute("""
SELECT
  MIN(prior_appt_count) AS min_prior,
  MAX(prior_appt_count) AS max_prior,
  AVG(prior_appt_count) AS mean_prior
FROM gold_appointments_features_v1
""").df()


,min_prior,max_prior,mean_prior
0,0,87,1.270223


### Export an updated feature contract (for modeling + later scoring)

In [6]:
import json

id_cols = ["appointment_id","person_id"]
label_col = "label"
treatment_col = "sms_received"

feature_cols = [
    "age",
    "gender",
    "neighbourhood",
    "scholarship","hipertension","diabetes","alcoholism","handcap",
    "lead_time_days","lead_time_clipped","lead_time_log1p",
    "lead_time_bin",
    "appt_dow","appt_month","sched_dow","sched_month","sched_hour",
    "nbhd_n",
    "prior_appt_count"
]

numeric_cols = [
    "age",
    "scholarship","hipertension","diabetes","alcoholism","handcap",
    "lead_time_days","lead_time_clipped","lead_time_log1p",
    "appt_dow","appt_month","sched_dow","sched_month","sched_hour",
    "nbhd_n",
    "prior_appt_count"
]

categorical_cols = [
    "gender","neighbourhood","lead_time_bin"
]

contract = {
    "id_cols": id_cols,
    "label_col": label_col,
    "treatment_col": treatment_col,
    "feature_cols": feature_cols,
    "numeric_cols": numeric_cols,
    "categorical_cols": categorical_cols
}

art_dir = repo_root / "Day-13" / "artifacts"
art_dir.mkdir(parents=True, exist_ok=True)

contract_path = art_dir / "noshow_feature_contract_v1.json"
contract_path.write_text(json.dumps(contract, indent=2), encoding="utf-8")

print("Wrote:", contract_path)
contract


Wrote: C:\Users\sarfo\Dropbox\Courses\Data Science\30-days-of-data-science\Day-13\artifacts\noshow_feature_contract_v1.json


{'id_cols': ['appointment_id', 'person_id'],
 'label_col': 'label',
 'treatment_col': 'sms_received',
 'feature_cols': ['age',
  'gender',
  'neighbourhood',
  'scholarship',
  'hipertension',
  'diabetes',
  'alcoholism',
  'handcap',
  'lead_time_days',
  'lead_time_clipped',
  'lead_time_log1p',
  'lead_time_bin',
  'appt_dow',
  'appt_month',
  'sched_dow',
  'sched_month',
  'sched_hour',
  'nbhd_n',
  'prior_appt_count'],
 'numeric_cols': ['age',
  'scholarship',
  'hipertension',
  'diabetes',
  'alcoholism',
  'handcap',
  'lead_time_days',
  'lead_time_clipped',
  'lead_time_log1p',
  'appt_dow',
  'appt_month',
  'sched_dow',
  'sched_month',
  'sched_hour',
  'nbhd_n',
  'prior_appt_count'],
 'categorical_cols': ['gender', 'neighbourhood', 'lead_time_bin']}

### Create a new smoke input example for scoring

In [7]:
smoke = pd.DataFrame({
    "appointment_id":[1,2,3],
    "person_id":[101,102,103],
    "age":[25,60,41],
    "gender":["F","M","F"],
    "neighbourhood":["JARDIM DA PENHA","CENTRO","SANTA MARTHA"],
    "scholarship":[0,1,0],
    "hipertension":[0,1,0],
    "diabetes":[0,0,1],
    "alcoholism":[0,0,0],
    "handcap":[0,0,0],
    "lead_time_days":[10,3,0],
    "lead_time_clipped":[10,3,0],
    "lead_time_log1p":[None,None,None],   # leave blank; production pipeline can compute
    "lead_time_bin":["3_7","1_2","same_day"],
    "appt_dow":[2,3,4],
    "appt_month":[5,5,5],
    "sched_dow":[1,2,3],
    "sched_month":[5,5,5],
    "sched_hour":[9,14,8],
    "nbhd_n":[3877,3334,3131],
    "prior_appt_count":[0,2,1],
})

in_dir = repo_root / "Day-13" / "data"
(in_dir / "raw").mkdir(parents=True, exist_ok=True)

smoke_path = in_dir / "input_example_v1.csv"
smoke.to_csv(smoke_path, index=False)
print("Wrote:", smoke_path)
smoke.head()


Wrote: C:\Users\sarfo\Dropbox\Courses\Data Science\30-days-of-data-science\Day-13\data\input_example_v1.csv


,appointment_id,person_id,age,gender,neighbourhood,scholarship,hipertension,diabetes,alcoholism,handcap,...,lead_time_clipped,lead_time_log1p,lead_time_bin,appt_dow,appt_month,sched_dow,sched_month,sched_hour,nbhd_n,prior_appt_count
0,1,101,25,F,JARDIM DA PENHA,0,0,0,0,0,...,10,None,3_7,2,5,1,5,9,3877,0
1,2,102,60,M,CENTRO,1,1,0,0,0,...,3,None,1_2,3,5,2,5,14,3334,2
2,3,103,41,F,SANTA MARTHA,0,0,1,0,0,...,0,None,same_day,4,5,3,5,8,3131,1
